## Notes on the Data

#### Data Documentation:
<br>**Description**: Synthetic dataset from Gap Inc., representing a random sample of individual purchases from Q1 FY2020. Each row is a unique item purchased in an order
<br><br>

| **Feature** | **Description**    | **Sample Value(s)**  |
| ------- | -----------    | ------------- |
| OrderID | Unique identifier per transaction (7-digit) | DRW7C20   |
| CustomerID | Unique identifier per customer (5-digit) | KP441   |
| ProductID  | Unique identifier per item (8-digit) | 13-817-239 |
| StoreID | Unique identifier per store (4-digit) | #4176 |
| OrderType | How purchase was completed  | InStore, HomeDelivery, Online |
| Timestamp | Timestamp of transaction (YYYY-MM-DD) | 2020-01-18 10:13:56	 |
| Brand | Which reporting segment of Gap Inc. bought from | Banana Republic |
| ItemSize | Size of item | XS, S, M, L, X, XL |
| ProductName | Name of item associated with item identifier | Pink Polo by Kanye |
| Collection | Which part of store | Denim Shop |
| Price | Listed price of item | $29.95 |
| ClearanceType | Type of clearance | Retail, Clearance, Final Sale |
| DiscountType | If Gap Card rewards was used | Reward points, Promotion, GapCash, Other |
| StoreName | Store name (i.e. Mall), or facility where online order was shipped from | Fair Oaks Mall |
| Location | State of store location | VA |

<br>

**Some comments about the data**: 

<br>In the real world, data comes from databases. Each transaction gets recorded as a observation, what item was purchased, at which store, by which customer. We can refer to these items by name, but there can be confusion if there's overlap. (i.e. Two customers with the same name, or two products with the same name)

<br>Instead, we can assign **unique identifiers** to each item, store, customer. Even if they have the same name, they'll be distinct in the eyes of the data. Since these are randomly assigned, they can look pretty ugly, but just know that **each ID is associated just one thing**

**Why we're using GroupBys**:

<br> Sometimes information in observations can be duplicated. Each row is a item that was purchased by some customer, on some day, at some store. That customer could have purchased *multiple items* in that transaction. To show that, we give each observation from the same transaction the *same OrderID*.

<br> This means that if we wanted some information at the transaction level, i.e. how many items were purchased per transaction, we could **first groupby each OrderID**, and then get some summary statistic, i.e 'count' to capture how many items are in each order.

<br> Now we have data where each row is a different order. To get the average number of items purchased per transaction, we now **average the grouped data**, and come up with a single number that summarizes that series. 

### Package & Data Imports

1. Import our usual packages, pandas and numpy. Use pandas to read in the CSV

2. Note that we'll have to pass in a new parameter, `sep=|`, to reflect that the raw data is stored a bit differently than usual

3. Check out the dimensions of the dataset, then take a look at the dataframe itself.

URL = `https://raw.githubusercontent.com/ishaandey/node/master/week-2/lab/gap.csv`

In [1]:
import pandas as pd
import numpy as np

In [34]:
df = pd.read_csv('https://raw.githubusercontent.com/ishaandey/node/master/week-2/lab/gap.csv', sep='|') # We're using a pipe instead of comma to seperate here

In [35]:
print(df.shape)
df.sample(2)

(4031, 14)


,OrderID,CustomerID,ProductID,StoreID,OrderType,Timestamp,Brand,ItemSize,ProductName,Collection,Price,ClearanceType,StoreName,Location
3844,8Q07Q75,UA313,76-558-812,#6658,InStore,2020-03-25 06:10:45,Banana Republic,M,Yes-I-am-Experiencing-Menopause-Thank-You-for-...,Women's Bottoms,34.95,FullRetail,Stony Point Fashion Park,VA
343,66Z341W,PD193,10-966-490,#9033,HomeDelivery,2020-02-23 22:35:16,Banana Republic,M,Tan Slacks for Serious Press Conference,Men's Bottoms,138.97,FinalSale,Dulles Town Center,VA


## Collection Trends

#### Questions (Easy):
Let's take a look at total sales in various collections:
1. First, subset down to the *Kids Tops* collection. Grab the price column, then take the sum of that. How much did Kids Tops sales total to?
2. What about total sales for the Denim Shop collection? For Accessories? 


<br>Since it's getting pretty tedious to look for total sales in each collection, let's automate this by taking the sum of sales in each collection. To do so:
3. Group the dataframe by Collection, grab the price column, then aggregate the rows by taking a sum across each collection.
4. Do your answers here line up with what we found by the subset method?

In [5]:
df[df.Collection == 'Kids Tops'].Price.sum() #1

13095.460000000001

In [6]:
df[df.Collection == 'Denim Shop'].Price.sum() #2

19796.47

In [7]:
df[df.Collection == 'Accessories'].Price.sum() #2

6139.860000000001

In [10]:
df.groupby('Collection').Price.agg('sum') # 3

Collection
Accessories         6139.86
Denim Shop         19796.47
Kids Bottoms        4746.85
Kids Tops          13095.46
Men's Bottoms      28907.40
Men's Tops         38469.18
Women's Bottoms    36413.43
Women's Tops       30972.46
Name: Price, dtype: float64

In [16]:
print(sum_by_collection['Kids Tops']) # 4
print(kids_tops_sales)

13095.460000000014
13095.460000000001


## Product Trends

#### Questions (Easy):
Let's take a look at some product trends:
1. How many unique products are there? 
2. What different collections are there?
3. What are top 10 best selling items (in terms of sale frequency)?
4. How many times was the `N-A-P Logo Branded Sweater` purchased?
<br>

#### Tips:
Functions you'll probably wanna use: `.unique()`, `.nunique()`, `.value_counts()` 

In [19]:
df.ProductName.nunique() # 1 

46

In [20]:
df.Collection.unique() # 2

array(['Accessories', 'Kids Tops', "Men's Bottoms", 'Denim Shop',
       "Men's Tops", "Women's Bottoms", "Women's Tops", 'Kids Bottoms'],
      dtype=object)

In [25]:
df.ProductName.value_counts().head() #3

Tan Slacks for Serious Press Conference                  173
Acid-Washed Low-Rise Jeans with LSD-tab-sized pockets    172
Tan Suit Jacket for Casual Press Conference              150
Let's-Wear-White-After-Labor-Day 3/4 Sleeve Blazer       133
Sun.png Summer Collection Sundress                       126
Name: ProductName, dtype: int64

In [31]:
df.ProductName.value_counts()['N-A-P Logo Branded Sweater']
# or
len(df[df.ProductName == 'N-A-P Logo Branded Sweater']) #4

106

## Segment Trends

#### Questions (Easy - Medium):
Let's take a look at some breakdowns by business segment, or brand.
1. What were *total sales* by brand (in terms of $)?


<br>Since a customer can buy multiple items in one transaction, or "order", use *OrderID* to prevent double counting.
2. What  were the total number of *unique* orders per brand? 
3. How many orders of each type (`OrderType`) were completed?
4. Could we break this down by brand?


#### Tips:
- First GroupBy, then Aggregate! `.groupby(by=)`, `.agg(func=)`
    - Google possible aggregation arguments, like sum, average, or number of unique items
- For question 4, use **multi-level groupbys**, i.e. pass a `list` of columns on which to groupby, instead of just one string.  

In [47]:
df.groupby('Brand').Price.agg('sum') #1

Brand
Banana Republic    120011.82
Gap                 58529.29
Name: Price, dtype: float64

In [48]:
df.groupby('Brand').OrderID.agg('nunique') #2

Brand
Banana Republic    1349
Gap                 823
Name: OrderID, dtype: int64

In [49]:
df.groupby('OrderType').OrderID.agg('nunique') #3

OrderType
HomeDelivery     576
InStore         1361
StorePickup      235
Name: OrderID, dtype: int64

In [50]:
df.groupby(['OrderType','Brand']).OrderID.agg('nunique') #4

OrderType     Brand          
HomeDelivery  Banana Republic    476
              Gap                100
InStore       Banana Republic    737
              Gap                624
StorePickup   Banana Republic    136
              Gap                 99
Name: OrderID, dtype: int64

## Store Trends

#### Questions (Medium):
Almost done! Think at the store level:
1. Which stores have the lowest sales (in $)? Save its StoreID to a varable, `bad_store`
2. How many orders does that store have? (Hint: Use `.index` to capture the key of a pd.Series object)
3. What is the maximum number of HomeDelivery orders that any one store got?
<br>

#### Tips:
- We're just chaining things together. Subset, then groupby, then pull a column, then aggregate, then apply a function, etc.

In [105]:
# 1

bad_store = df.groupby('StoreID').Price.agg('sum').sort_values(ascending=False).tail(1)
bad_store

StoreID
#1812    4325.72
Name: Price, dtype: float64

In [98]:
# 2
df.groupby('StoreID').OrderID.agg('nunique')[bad_store.index]

StoreID
#1812    68
Name: OrderID, dtype: int64

In [104]:
# 3
df[df.OrderType == 'HomeDelivery'].groupby('StoreID').OrderID.agg('nunique').max()

49

## Customer Trends

#### Questions (Medium - Hard):
Let's look at the customers. A couple of questions that management wants us to answer:
1. At Gap, how many different stores did each customer shop at, on average? 
2. On average, how much does each customer spend per transaction? 
3. Does this value differ when broken down by brand? Which store do customers spend more at?
<br>

#### Tips:
- In general: Think about chaining! Subset, then groupby, then pull a column, then aggregate, then apply a function, etc.
- For #1: Since each customer is assigned a unique ID, we wanna get our data at a *per-customer* level. In other words, we'll group by CustomerID first.
- For #2: Try to sum up how much *each customer*, at *each order*, spent in total. Then take the average of this.
- For #3: Add another level to your previous solution, but then *group* up to the brand level again to consolidate all the customer-orders into each brand

In [78]:
# 1
df[df.Brand == 'Gap'].groupby('CustomerID').StoreID.agg('nunique').mean()

1.4915254237288136

In [79]:
# 2
df.groupby(['CustomerID', 'OrderID']).Price.agg('sum').mean()

82.20124769797421

In [80]:
# 3
df.groupby(['Brand','CustomerID', 'OrderID']).Price.agg('sum').groupby('Brand').mean()

Brand
Banana Republic    88.963543
Gap                71.116999
Name: Price, dtype: float64